In [1]:
import requests
from xml.etree import ElementTree as ET

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
from matplotlib import rcParams
import seaborn as sns
from scipy.stats import norm
from scipy.stats import shapiro
from scipy.stats import anderson
import statsmodels.api as sm
from datetime import datetime
from datetime import timedelta
import os   

plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

## DATASET

In [28]:
raw_fog = pd.read_csv(f'C:/Users/USER/DATA WORK PLACE/2. DATA STUDY/240508 WEATHER/data/fog_train.csv', encoding = 'euc-kr')
fog_test = pd.read_csv(f'C:/Users/USER/DATA WORK PLACE/2. DATA STUDY/240508 WEATHER/data/fog_test.csv', encoding = 'euc-kr')

raw_fog

,Unnamed: 0,fog_train.year,fog_train.month,fog_train.day,fog_train.time,fog_train.minute,fog_train.stn_id,fog_train.ws10_deg,fog_train.ws10_ms,fog_train.ta,fog_train.re,fog_train.hm,fog_train.sun10,fog_train.ts,fog_train.vis1,fog_train.class
0,1,I,1,1,0,10,AA,0.0,0.0,-6.4,0.0,38.9,0.0,-2.8,20000,4
1,2,I,1,1,0,20,AA,0.0,0.0,-6.3,0.0,37.9,0.0,-2.7,20000,4
2,3,I,1,1,0,30,AA,0.0,0.0,-6.3,0.0,40.0,0.0,-2.6,20000,4
3,4,I,1,1,0,40,AA,27.8,0.4,-6.2,0.0,39.5,0.0,-2.6,20000,4
4,5,I,1,1,0,50,AA,59.7,0.5,-6.1,0.0,39.8,0.0,-2.5,20000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3156454,3156455,K,12,31,23,10,EC,305.3,7.7,2.5,0.0,50.2,0.0,-0.9,20000,4
3156455,3156456,K,12,31,23,20,EC,293.8,5.7,2.3,0.0,50.1,0.0,-1.3,20000,4
3156456,3156457,K,12,31,23,30,EC,274.2,4.9,2.2,0.0,51.0,0.0,-1.4,20000,4
3156457,3156458,K,12,31,23,40,EC,270.3,4.6,2.1,0.0,51.7,0.0,-1.6,20000,4


In [29]:
raw_fog[raw_fog['fog_train.class'] < -90]

,Unnamed: 0,fog_train.year,fog_train.month,fog_train.day,fog_train.time,fog_train.minute,fog_train.stn_id,fog_train.ws10_deg,fog_train.ws10_ms,fog_train.ta,fog_train.re,fog_train.hm,fog_train.sun10,fog_train.ts,fog_train.vis1,fog_train.class
7645,7646,I,2,23,2,20,AA,272.0,1.3,-0.9,0.0,61.1,0.00,-1.0,-99,-99
7780,7781,I,2,24,0,50,AA,140.7,1.2,2.1,0.0,59.5,0.00,0.8,-99,-99
16053,16054,I,4,21,11,40,AA,271.7,7.0,9.2,0.0,42.8,0.32,16.7,-99,-99
16054,16055,I,4,21,11,50,AA,276.4,6.0,9.3,0.0,43.4,0.26,19.0,-99,-99
16055,16056,I,4,21,12,0,AA,268.4,5.9,9.5,0.0,41.7,0.48,21.1,-99,-99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3155926,3155927,K,12,28,7,10,EC,307.9,4.3,2.0,0.0,62.6,0.00,-1.4,-99,-99
3155961,3155962,K,12,28,13,0,EC,294.5,6.1,6.7,0.0,41.3,0.32,15.2,-99,-99
3156100,3156101,K,12,29,12,10,EC,16.3,3.5,4.7,0.0,43.2,0.34,14.6,-99,-99
3156135,3156136,K,12,29,18,0,EC,314.4,8.8,3.5,0.0,42.0,0.00,1.6,-99,-99


In [30]:
def raw_data_preprocessing(dataset):

    fog_data = dataset.copy()

    fog_data = fog_data.drop(['Unnamed: 0'], axis = 1)

    fog_data = fog_data.replace(-99.00, np.NaN)
    fog_data = fog_data.replace(-99.90, np.NaN)

    columns = fog_data.columns
    columns = [column.replace('fog_train.', "") for column in columns]
    fog_data.columns = columns

    return fog_data

In [31]:
fog_data = raw_data_preprocessing(raw_fog)
fog_test = raw_data_preprocessing(fog_test)
fog_data

,year,month,day,time,minute,stn_id,ws10_deg,ws10_ms,ta,re,hm,sun10,ts,vis1,class
0,I,1,1,0,10,AA,0.0,0.0,-6.4,0.0,38.9,0.0,-2.8,20000.0,4.0
1,I,1,1,0,20,AA,0.0,0.0,-6.3,0.0,37.9,0.0,-2.7,20000.0,4.0
2,I,1,1,0,30,AA,0.0,0.0,-6.3,0.0,40.0,0.0,-2.6,20000.0,4.0
3,I,1,1,0,40,AA,27.8,0.4,-6.2,0.0,39.5,0.0,-2.6,20000.0,4.0
4,I,1,1,0,50,AA,59.7,0.5,-6.1,0.0,39.8,0.0,-2.5,20000.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3156454,K,12,31,23,10,EC,305.3,7.7,2.5,0.0,50.2,0.0,-0.9,20000.0,4.0
3156455,K,12,31,23,20,EC,293.8,5.7,2.3,0.0,50.1,0.0,-1.3,20000.0,4.0
3156456,K,12,31,23,30,EC,274.2,4.9,2.2,0.0,51.0,0.0,-1.4,20000.0,4.0
3156457,K,12,31,23,40,EC,270.3,4.6,2.1,0.0,51.7,0.0,-1.6,20000.0,4.0


## REGRESSION

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import *

### DATA preprocessing

#### basic

In [35]:
model_data = fog_data.copy()
model_data = model_data.drop(['year', 'month', 'day', 'time', 'minute', 'stn_id', 'class'], axis = 1)
model_data = model_data.dropna()
model_data

,ws10_deg,ws10_ms,ta,re,hm,sun10,ts,vis1
0,0.0,0.0,-6.4,0.0,38.9,0.0,-2.8,20000.0
1,0.0,0.0,-6.3,0.0,37.9,0.0,-2.7,20000.0
2,0.0,0.0,-6.3,0.0,40.0,0.0,-2.6,20000.0
3,27.8,0.4,-6.2,0.0,39.5,0.0,-2.6,20000.0
4,59.7,0.5,-6.1,0.0,39.8,0.0,-2.5,20000.0
...,...,...,...,...,...,...,...,...
3156454,305.3,7.7,2.5,0.0,50.2,0.0,-0.9,20000.0
3156455,293.8,5.7,2.3,0.0,50.1,0.0,-1.3,20000.0
3156456,274.2,4.9,2.2,0.0,51.0,0.0,-1.4,20000.0
3156457,270.3,4.6,2.1,0.0,51.7,0.0,-1.6,20000.0


In [42]:
model_test = fog_test.drop(['year', 'month', 'day', 'time', 'minute', 'stn_id', 'class'], axis = 1)

#### split

In [49]:
dataset_preprocessed = model_data.copy()

x = dataset_preprocessed.drop(['vis1'], axis = 1)
y = dataset_preprocessed['vis1']

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size= 0.3, random_state=2077)
# stratify는 데이터 클래스의 비율을 일정한 수준으로 유지한채 데이터를 스플릿하는 함수임

In [50]:
x_test, y_test = model_test.drop(['vis1'], axis = 1), fog_test['vis1']

#### scaling

In [51]:
scaler = MinMaxScaler()

scaler.fit(x_train)

scaled_x_train = pd.DataFrame(data = scaler.transform(x_train), columns = x_train.columns)
scaled_x_valid = pd.DataFrame(data = scaler.transform(x_valid), columns = x_valid.columns)
scaled_x_test = pd.DataFrame(data = scaler.transform(x_test), columns = x_test.columns)

### modeling and test

#### XGB

In [52]:
params = {
    'n_estimators' : (50, 100, 200),
    'learning_rate' : (0.1, 0.3, 0.5),
    'max_depth' : (3, 5, 8, 10),
    'gamma' : (0, 1, 5, 10),
    'importance_type' : ('gain', 'weight', 'cover'),
    'reg_lambda' : (0.2, 0.5, 0.7, 1),
    'subsample' : (0.1, 0.5, 1)}

xgb_model = XGBRegressor(random_state = 2077, n_jobs = -1)
grid_cv = GridSearchCV(xgb_model, param_grid = params, cv = 3, n_jobs = -1)
grid_cv.fit(scaled_x_train, y_train)

print('최적 하이퍼 파라미터:', grid_cv.best_params_)
print('최적 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

In [ ]:
xgb_model = XGBRegressor(n_estimators=100, # 붓스트랩 샘플 개수 또는 base_estimator 개수
                         learning_rate=0.1, 
                         max_depth=3, # 개별 나무의 최대 깊이
                         gamma = 0,
                         importance_type='gain',
                         reg_lambda = 0.5, ## tuning parameter of l2 penalty
                         random_state = 2077,
                         subsample = 1,
                         n_jobs = -1)

xgb_model.fit(scaled_x_train, y_train)
xgb_model

#### XGB 예측 및 평가

In [ ]:
xgb_predict = xgb_model.predict(scaled_x_valid)
xgb_RMSE = np.sqrt(np.mean((y_valid - xgb_predict) ** 2))
xgb_RMSE

In [ ]:
df = pd.DataFrame()
df['feature'] = list(scaled_x_train.columns)
df['importance'] = xgb_model.feature_importances_
df.sort_values(by = 'importance', ascending= True, inplace=True)

plt.barh(y= 'feature', width = 'importance', data = df)
plt.title("XGB feature importance", fontsize = 20)
plt.show()

#### LightGBM

In [ ]:
pip install lightgbm

In [ ]:
params = {
    'n_estimators' : (50, 100, 200),
    'learning_rate' : (0.1, 0.3, 0.5),
    'max_depth' : (3, 5, 8, 10),
    'lambda_l1' : (10, 30, 50, 70),
    'lambda_l2' : (10, 30, 50, 70),
    'min_gain_to_split' : (3, 5, 7),
    'subsample' : (0.1, 0.3, 0.6, 0.9)}

lgb_model = LGBMRegressor(random_state = 2077, n_jobs = -1)
grid_cv = GridSearchCV(lgb_model, param_grid = params, cv = 3, n_jobs = -1)
grid_cv.fit(scaled_x_train, y_train)

print('최적 하이퍼 파라미터:', grid_cv.best_params_)
print('최적 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

In [ ]:
lgb_model = LGBMRegressor(n_estimators = 50,
                          learning_rate = 0.1,
                          lambda_l1 = 10,
                          lambda_l2 = 10,
                          max_depth = 8,
                          min_gain_to_split = 3,
                          subsample = 0.1,
                          random_state = 2077,
                          n_jobs = -1)

lgb_model.fit(scaled_x_train, y_train)
lgb_model

#### LightGBM 예측 및 평가

In [ ]:
lgb_predict = lgb_model.predict(scaled_x_valid)
lgb_RMSE = np.sqrt(np.mean((y_valid - lgb_predict) ** 2))
lgb_RMSE

In [ ]:
df = pd.DataFrame()
df['feature'] = list(scaled_x_train.columns)
df['importance'] = lgb_model.feature_importances_
df.sort_values(by = 'importance', ascending= True, inplace=True)

plt.barh(y= 'feature', width = 'importance', data = df)
plt.title("LGB feature importance", fontsize = 20)
plt.show()

#### Random Forest

In [ ]:
params = {
    'n_estimators':(50, 100, 200),
    'max_depth' : (3, 5, 8, 10),
    'min_samples_leaf' : (3, 5, 8, 10),
    'min_samples_split' : (3, 5, 10, 15)}

rf = RandomForestRegressor(random_state = 2077, n_jobs = -1)
grid_cv = GridSearchCV(rf, param_grid = params, cv = 3, n_jobs = -1)
grid_cv.fit(scaled_x_train, y_train)

print('최적 하이퍼 파라미터:', grid_cv.best_params_)
print('최적 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

In [ ]:
rf = RandomForestRegressor(random_state = 2077, 
                           max_depth = 10, 
                           min_samples_leaf = 10, 
                           min_samples_split = 10,
                           n_estimators = 200,
                           n_jobs = -1)

rf.fit(scaled_x_train, y_train)

#### Random Forest 예측 및 평가

In [ ]:
rf_predict = rf.predict(scaled_x_valid)
rf_RMSE = np.sqrt(np.mean((y_valid - rf_predict) ** 2))
rf_RMSE

In [ ]:
df = pd.DataFrame()
df['feature'] = list(scaled_x_train.columns)
df['importance'] = rf.feature_importances_
df.sort_values(by = 'importance', ascending= True, inplace=True)

plt.barh(y= 'feature', width = 'importance', data = df)
plt.title("RF feature importance", fontsize = 20)
plt.show()

#### 다중회귀

In [ ]:
lr_model = LinearRegression()
lr_model.fit(scaled_x_train, y_train)